In [1]:
import numpy as np
import pandas as pd
import os
import emcee
from multiprocessing import Pool, cpu_count
import scipy.integrate as integrate

# Define file paths
base_directory = 'C:\\Users\\Lorenzo\\Desktop\\University Documents\\Unito Fisica\\Tesi Magistrale\\Code\\save'
covariance_directory = 'C:\\Users\\Lorenzo\\Desktop\\University Documents\\Unito Fisica\\Tesi Magistrale\\Code\\save\\covariance_matrix'

# Define constants
G = 4.30091e-6

# Define directories and variables
star_number = 100
anisotropy_type = 'iso'
profile_type = 'core'
space_type = 'cartesian'
print_matrix = 0  # Set this to choose which covariance matrix to print



nwalker = 10 #50
ndim = 3 #this dfines the number of model parameters that required to be constrained.
niter = 100 #500


In [2]:
#Functions

#Create mockdata
def load_data(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, names=['x', 'y', 'z', 'vx', 'vy', 'vz'])

    return data.to_numpy()  # Convert the DataFrame to a NumPy array, 
        
#Create cov matrixes
def load_covariance_matrix(star_index, anisotropy_type,star_number):


    file_name = f'Cij_cartesian_stars_{star_index}_{profile_type}_beta_{anisotropy_type}.txt'
    covariance_star_path = file_path = os.path.join(covariance_directory, f'star{star_number}')
    file_path = os.path.join(covariance_star_path, file_name)
    
    matrix = np.loadtxt(file_path)
    return matrix

# Calculate the sigma_vt^2 value for a given covariance matrix index
def calculate_sigma_vt2(matrix_index, covariance_matrices): #matrix index gli passero poi star_index
    
    covariance_matrix = covariance_matrices[matrix_index]
    
    sigma_vtheta_vtheta = covariance_matrix[1, 1]
    sigma_vphi_vphi = covariance_matrix[2, 2]
    
    sigma_vt2 = (sigma_vtheta_vtheta + sigma_vphi_vphi) / 2
    return sigma_vt2

#Create a 3x3 instrumental covariance matrix with specified sigma_v.
def create_S_instrumental(sigma_v):

    S_instrumental = np.zeros((3, 3))
    np.fill_diagonal(S_instrumental, sigma_v**2)
    return S_instrumental

def calculate_mu_v(mock_data):
    
    avg_vx = np.mean(mock_data[3])
    avg_vy = np.mean(mock_data[4])
    avg_vz = np.mean(mock_data[5])
    
    mu_v = np.array([[avg_vx], [avg_vy], [avg_vz]])
    return mu_v

def rho_DM(r, rho0, rs, gamma):
        return rho0 / ((r / rs)**gamma * (1 + r / rs)**(3 - gamma))
    
def integrand(r, rho0, rs, gamma):
        return r**2 * rho_DM(r, rho0, rs, gamma)


def DM_profile_model(mock_data, params, star_index):

    # Unpack the parameters
    rho0, rs, gamma = params

    r = np.sqrt(mock_data[star_index][0]**2 + mock_data[star_index][1]**2 + mock_data[star_index][2]**2)

    rho_DM(r, rho0, rs, gamma)
    integrand(r, rho0, rs, gamma)    

    integral, _ = integrate.quad(integrand, 0, r, args = (rho0, rs, gamma))
    V_squared = (4 * np.pi * G / r) * integral
        
    V_r = np.sqrt(V_squared)
    
    return V_r


def p_v_given_r(star_index, mu_v, S_instrumental, covariance_matrix, ndim, mock_data, params):

    V_r = DM_profile_model(mock_data, params, star_index) #eh ma sta velocità è un numero non un vettore

    # Calculate relative velocity vector
    v_relative = V_r - np.linalg.norm(mu_v)
    # Calculate S_total which is the sum of S_instrumental and covariance_matrix
    S_total = S_instrumental + covariance_matrix

    log_numerator = (-0.5 * np.dot(np.dot(v_relative, np.linalg.inv(S_total)), v_relative))
    log_denominator = 0.5*np.log10(((2 * np.pi)**ndim)*(np.linalg.det(S_total)))

    return log_numerator - log_denominator



#Likelyhood
def log_likelyhood(params, mock_data, covariance_matrices, mu_v, S_instrumental, ndim, star_number):
    L_params = 0

    for star_index in range(star_number):
    
        covariance_matrix = covariance_matrices[star_index]
        position_cartesian = mock_data[star_index][:3]
        #v_cartesian = mock_data[star_index][3:]

        prob_given_r = p_v_given_r(star_index, mu_v, S_instrumental, covariance_matrix, ndim, mock_data, params)

        L_params += prob_given_r

    return L_params

def trylog(params, mock_data, covariance_matrices, mu_v, S_instrumental, ndim, star_number):
     
     rho0, rs, gamma = params
     mod = (rho0+rs+gamma)/(rho0^2+rs^2+gamma^2)
     return mod
     
    

    




In [3]:
#Main

#Load the mock data

print('dioschifo')

mock_data_file = f'Mockdata_{space_type}_{star_number}_stars_{profile_type}_beta_{anisotropy_type}.txt'
mock_data_path = os.path.join(base_directory, mock_data_file)

mock_data = load_data(mock_data_path) #array [][] che contiene i dati
# Print the first row of mock data
print("First row of mock data:")
print(mock_data[0])

# Load covariance matrices
#covariance_directory = os.path.join(base_directory, f'covariance_matrix\\star{star_number}')
covariance_matrices = []

print('diomerda')

for star_index in range(star_number):
    matrix = load_covariance_matrix(star_index, anisotropy_type, star_number)
    covariance_matrices.append(matrix) #array di matrici di covarianza

print('diporco')

# Print the first covariance matrix

print(covariance_matrices[0])

rho0 = 2.284
rs = -0.824
gamma = 0
#beta = 0

params = [rho0, rs, gamma]
sigma_v = 0  # Set this to 0, 1, or 5 as needed

mu_v = calculate_mu_v(mock_data)
S_instrumental = create_S_instrumental(sigma_v)

print('diofrocio')

#p0 = np.random.uniform(low=[-3, -2, -3, -10], high=[5, 3, 3, 1], size=(nwalker, ndim)) #vettore con inizializzazione randomica dei parametri con beta
p0 = np.random.uniform(low=[-3, -2, -3], high=[5, 3, 3], size=(nwalker, ndim)) #vettore con inizializzazione randomica dei parametri

print('diopene')

with Pool(processes=4) as pool:
    
    sampler = emcee.EnsembleSampler(nwalker, ndim, trylog, pool=pool)
    sampler.run_mcmc(p0, niter, progress = True)

print(params)



dioschifo
First row of mock data:
[ 0.23216535 -0.02628695 -0.18520756  1.05255798 -6.33599615 -0.69663847]
diomerda
diporco
[[67.496296  0.        0.      ]
 [ 0.       67.496296  0.      ]
 [ 0.        0.       67.496296]]
diofrocio
diopene
